In [1]:
import pandas as pd
import numpy as np

# Read data

XYtr = pd.read_csv('data/XYtr.csv')



XYtr1 = XYtr.copy() # from stackoverflow
XYtr1['cdate'] = pd.to_datetime(XYtr1['cdate']).values.astype(np.float64)/8.64e+13

Ytr1 = XYtr1['total'].copy()
Xtr1 = XYtr1[['X.sales', 'cdate', 'fee1', 'fee2']].copy()

Xtr1 = Xtr1.astype(np.float64)

Xtr1 = Xtr1.fillna(0)

pred = pd.read_csv('data/pred.csv')

Xte = pd.read_csv('data/Xte.csv')

Xte1 = Xte.copy()
Xte1['cdate'] = pd.to_datetime(Xte1['cdate']).values.astype(np.float64)/8.64e+13
Xte1 = Xte1[['X.sales', 'cdate', 'fee1', 'fee2']].copy()
Xte1 = Xte1.astype(np.float64)
Xte1 = Xte1.fillna(0)

In [2]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

from sklearn.linear_model import SGDRegressor

model3 = SGDRegressor(loss = 'epsilon_insensitive', alpha = 0, epsilon = 0).fit(Xtr1, Ytr1)

C:\Users\Benson\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1503: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


In [3]:
pred4 = pred.copy()
pred4['total'] = model3.predict(Xte1)
pred4.to_csv('data/pred4.csv', index = False)

In [2]:
Xtr_fi = pd.read_csv('data/Xtr_fi.csv')
Xtr_fi1 = Xtr_fi.copy()
Xtr_fi1.drop('id',1, inplace = True)
Xtr_fi1 = Xtr_fi1.fillna(0)

Xte_fi = pd.read_csv('data/Xte_fi.csv')
Xte_fi1 = Xte_fi.copy()
Xte_fi1.drop('id',1, inplace = True)
Xte_fi1 = Xte_fi1.fillna(0)

Xtr_ft = pd.read_csv('data/XYtr_ft.csv')
Xtr_ft1 = Xtr_ft.copy()
Xtr_ft1.drop('id',1, inplace = True)
Xtr_ft1 = Xtr_ft1.fillna(0)

Xte_ft = pd.read_csv('data/Xte_ft.csv')
Xte_ft1 = Xte_ft.copy()
Xte_ft1.drop('id',1, inplace = True)
Xte_ft1 = Xte_ft1.fillna(0)

C:\Users\Benson\AppData\Local\Temp/ipykernel_13268/2976609976.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Xtr_fi1.drop('id',1, inplace = True)
C:\Users\Benson\AppData\Local\Temp/ipykernel_13268/2976609976.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Xte_fi1.drop('id',1, inplace = True)
C:\Users\Benson\AppData\Local\Temp/ipykernel_13268/2976609976.py:13: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Xtr_ft1.drop('id',1, inplace = True)
C:\Users\Benson\AppData\Local\Temp/ipykernel_13268/2976609976.py:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  Xte_ft1.drop('id',1, inplace = True)


In [3]:
Xtr5 = pd.concat([Xtr1, Xtr_fi1, Xtr_ft1], axis = 1)
Xte5 = pd.concat([Xte1, Xte_fi1, Xte_ft1], axis = 1)

In [30]:
Xtr6 = Xtr5.copy()
Xte6 = Xte5.copy()

In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

Xtr5 = scaler.fit_transform(Xtr5)
Xte5 = scaler.transform(Xte5)

from sklearn.linear_model import SGDRegressor

model5 = SGDRegressor(loss = 'epsilon_insensitive', alpha = 0, epsilon = 0).fit(Xtr5, Ytr1)
pred5 = pred.copy()
pred5['total'] = model5.predict(Xte5)
pred5.to_csv('data/pred5.csv', index = False)


In [7]:
# Taking the top 90% frequency of words

from sklearn.feature_extraction.text import CountVectorizer

XYtr2 = XYtr.copy()
Xte2 = Xte.copy()

XYtr2['description'] = XYtr2['description'].fillna("NAN")
Xte2['description'] = Xte2['description'].fillna("NAN")
corpus = list(XYtr2['description'])+list(Xte2['description'])

# https://stackoverflow.com/questions/62777131/how-to-remove-frequent-infrequent-features-from-sklearn-countvectorizer 
# Code to remove top X% most frequent words (use this to remove stop words).
vect = CountVectorizer()
bow = vect.fit_transform(corpus)
total_features = len(vect.vocabulary_)

top_vect = CountVectorizer(max_features=int(total_features * 0.1))
top_bow = top_vect.fit_transform(corpus)

# Create a list of (term, frequency) tuples sorted by their frequency
sum_words = bow.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in vect.vocabulary_.items()]
words_freq = sorted(words_freq, key = lambda x: x[1], reverse = True)

# Keep only the terms in a list
vocabulary, _ = zip(*words_freq[:int(total_features*0.9)])
vocabulary = list(vocabulary)

bottom_vect = CountVectorizer(vocabulary=vocabulary)
bottom_bow = bottom_vect.fit_transform(corpus)

In [8]:
Xtr5

array([[-0.07386261, -0.78120744, -0.5305535 , ..., -0.39804996,
        -0.41193557, -0.32364991],
       [ 0.27845913,  1.08654395, -0.5305535 , ..., -0.35177543,
        -0.36611893, -0.27457164],
       [-0.05667618, -0.78120744,  0.9514467 , ..., -0.40077064,
        -0.4146296 , -0.32653573],
       ...,
       [-0.06526939, -0.74038228, -1.2715536 , ..., -0.40777126,
        -0.42156164,  4.41715886],
       [-0.06526939,  1.86562422, -0.67875352, ..., -0.39169407,
        -0.40564195,  1.45621467],
       [-0.04808297,  1.02870829,  1.6924468 , ..., -0.18847481,
         2.05947801, -0.10135478]])

In [33]:
from sklearn.ensemble import RandomForestRegressor

rnd_clf = RandomForestRegressor(n_estimators = 500, max_leaf_nodes = 16, n_jobs = -1)

rnd_clf.fit(Xtr6,Ytr1)

pred6 = pred.copy()
pred6['total'] = rnd_clf.predict(Xte5)
# pred6.to_csv('data/pred6.csv', index = False)
for name, score in zip(Xtr6, rnd_clf.feature_importances_):
    print(name, score)

# pred6

X.sales 0.015191198550367097
cdate 0.03218056458087197
fee1 0.061344678121509406
fee2 0.03505050253250287
fi1 0.001702073712937516
fi2 0.0002708624632651714
fi3 0.1311018579618168
fi4 0.021974400643703033
fi5 0.08496060427799958
fi6 0.2146038960168166
fi7 0.06932031029369573
FT0000 0.04373871388611657
FT0001 0.010423127064004549
FT0002 0.024210332236395682
FT0003 0.006390672005282188
FT0004 0.04755858988623157
FT0005 0.01970404599253934
FT0006 0.0241397370900566
FT0007 0.020619029534109996
FT0008 0.11310034445066676
FT0009 0.02241445869911095


C:\Users\Benson\anaconda3\envs\tf\lib\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [10]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth = 2, n_estimators = 3, learning_rate = 1.0)

gbrt.fit(Xtr5,Ytr1)

pred7 = pred.copy()
pred7['total'] = gbrt.predict(Xte5)
pred7.to_csv('data/pred7.csv', index = False)

In [11]:
import xgboost

xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(Xtr5,Ytr1)

pred8 = pred.copy()
pred8['total'] = xgb_reg.predict(Xte5)
pred8.to_csv('data/pred8.csv', index = False)

In [12]:
import tensorflow as tf
from tensorflow import keras

Xtr5.shape
#x_tr = np.reshape(Xtr5,(1,21))
#x_tr.dtypes

(6914, 21)

In [16]:
Xtr5

array([[-0.07386261, -0.78120744, -0.5305535 , ..., -0.39804996,
        -0.41193557, -0.32364991],
       [ 0.27845913,  1.08654395, -0.5305535 , ..., -0.35177543,
        -0.36611893, -0.27457164],
       [-0.05667618, -0.78120744,  0.9514467 , ..., -0.40077064,
        -0.4146296 , -0.32653573],
       ...,
       [-0.06526939, -0.74038228, -1.2715536 , ..., -0.40777126,
        -0.42156164,  4.41715886],
       [-0.06526939,  1.86562422, -0.67875352, ..., -0.39169407,
        -0.40564195,  1.45621467],
       [-0.04808297,  1.02870829,  1.6924468 , ..., -0.18847481,
         2.05947801, -0.10135478]])

In [ ]:
# MAE calculation
# sum(abs(y*-y))/n

In [13]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[6914, 21]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy",
optimizer="sgd", metrics=["accuracy"])

history = model.fit(Xtr5, Ytr1, epochs=30)

Epoch 1/30


ValueError: in user code:

    C:\Users\Benson\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    C:\Users\Benson\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Benson\anaconda3\envs\tf\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Benson\anaconda3\envs\tf\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Benson\anaconda3\envs\tf\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\Benson\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py:835 run_step  **
 